In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta

In [2]:
rds_connection_string = "root:12345678@127.0.0.1/fires"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [3]:
wildfire_df = pd.read_sql("select * from cleaned_wildfire_data",con=engine)


/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [4]:
wildfire_df = wildfire_df[['fire_id','fire_name', 'discovery_date', 'containment_date', 'size', 'class', 'fire_year']]



In [5]:
wildfire_df.head()

,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year
0,0,FOUNTAIN,2005-02-02,2005-02-02,0.10,A,2005
1,1,PIGEON,2004-05-12,2004-05-12,0.25,A,2004
2,2,SLACK,2004-05-31,2004-05-31,0.10,A,2004
3,3,DEER,2004-06-28,2004-07-03,0.10,A,2004
4,4,STEVENOT,2004-06-28,2004-07-03,0.10,A,2004


In [6]:
airquality_df = pd.read_sql("select * from aqi_data", con=engine)
airquality_df.head()


,date,overall_aqi,site_name,year
0,2014-01-01,109,Oakland West,2014
1,2014-01-02,80,Oakland West,2014
2,2014-01-03,80,Oakland West,2014
3,2014-01-04,67,Oakland West,2014
4,2014-01-05,56,Livermore,2014


In [7]:
merged_data_df = airquality_df.merge(wildfire_df,left_on='date', right_on="containment_date", how='left')

In [8]:
merged_data_df.head()

,date,overall_aqi,site_name,year,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year
0,2014-01-01,109,Oakland West,2014,171166.0,PIUMA,2014-01-01,2014-01-01,0.3,B,2014.0
1,2014-01-01,109,Oakland West,2014,171798.0,None,2014-01-01,2014-01-01,0.9,B,2014.0
2,2014-01-01,109,Oakland West,2014,172306.0,None,2014-01-01,2014-01-01,60.0,C,2014.0
3,2014-01-01,109,Oakland West,2014,172307.0,None,2014-01-01,2014-01-01,0.3,B,2014.0
4,2014-01-01,109,Oakland West,2014,172308.0,None,2014-01-01,2014-01-01,1.0,B,2014.0


In [9]:
# and write it to the database
merged_data_df.to_sql(name="fire_and_air_data", con=engine, if_exists="replace", index=False)

In [10]:
# confirm that there are wildfires with no name
wildfire_df[wildfire_df["containment_date"] == "2014-01-01"]

,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year
171166,171166,PIUMA,2014-01-01,2014-01-01,0.3,B,2014
171798,171798,None,2014-01-01,2014-01-01,0.9,B,2014
172306,172306,None,2014-01-01,2014-01-01,60.0,C,2014
172307,172307,None,2014-01-01,2014-01-01,0.3,B,2014
172308,172308,None,2014-01-01,2014-01-01,1.0,B,2014
172309,172309,None,2014-01-01,2014-01-01,1.0,B,2014
172573,172573,None,2014-01-01,2014-01-01,0.1,A,2014
172683,172683,None,2014-01-01,2014-01-01,0.1,A,2014
173659,173659,None,2014-01-01,2014-01-01,3.5,B,2014
173660,173660,None,2014-01-01,2014-01-01,0.3,B,2014
